In [1]:
import os
import sys
import pprint

base_dir = 'C:\Documents\Thesis_ssd\MasterThesis-2.0'
os.chdir(base_dir)

from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.BaselineHelperFunctions import BaselineHelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.DataGenerator import DataGenerator
from Classes.Modeling.Models import Models
from Classes.Modeling.RandomGridSearch import RandomGridSearch
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
import json

helper = BaselineHelperFunctions()
%load_ext tensorboard

In [2]:
num_classes = 2
loadData = LoadData(num_classes, isBalanced = True)
shuffle = False
full_ds, train_ds, val_ds, test_ds = loadData.getDatasets(shuffle = shuffle)
handler = DataHandler()
pp = pprint.PrettyPrinter(indent=4)
helper = BaselineHelperFunctions()

In [3]:
hyper_grid = {
        "batch_size" : [8, 16, 32, 64, 128, 256, 512],
        "epochs" : [1],
        "learning_rate" : [0.1, 0.01, 0.001, 0.0001, 0.00001],
        "optimizer" : ["adam", "rmsprop", "sgd"]
    }
model_grid = {
    "activation" : ["relu", "sigmoid", "softmax", "tanh"],
    "dropout_rate" : [0.5, 0.4, 0.3, 0.2, 0.1, 0.01, 0],
    "filters" : [11, 13, 15, 17, 19, 21, 23, 25],
    "kernel_size" : [3, 5, 7, 9, 11, 13],
    "l1_r" : [0.3, 0.2, 0.1, 0.01, 0.001, 0.0001],
    "l2_r" : [0.3, 0.2, 0.1, 0.01, 0.001, 0.0001],
    "output_layer_activation" : ["softmax", "sigmoid"],
    "padding" : ["same"],
    "start_neurons" : [8,16, 32, 64, 128, 256, 512, 1024]
}
"""
{'activation': 'softmax', 
 'dropout_rate': 0.1, 
 'filters': 25, 
 'kernel_size': 9, 
 'l1_r': 0.01, 
 'l2_r': 0.0001, 
 'output_layer_activation': 'softmax', 
 'padding': 'same', 
 'start_neurons': 32}
"""


model_nr = 5

test_mode = True
use_scaler = False
use_noise_augmentor = False
detrend = False
use_minmax = False
use_highpass = False
highpass_freq = 0.1

n_picks = 2

use_tensorboard = False
use_liveplots = False
use_custom_callback = False
use_early_stopping = True

In [4]:
randomGridSearch = RandomGridSearch(train_ds, val_ds, test_ds, model_nr, test_mode, detrend,
                                    use_scaler, use_noise_augmentor, use_minmax, use_highpass, n_picks, 
                                    hyper_grid = hyper_grid, model_grid = model_grid, 
                                    num_classes = num_classes, use_tensorboard = use_tensorboard,
                                    use_liveplots = use_liveplots, use_custom_callback = use_custom_callback,
                                    use_early_stopping = use_early_stopping, highpass_freq = highpass_freq)

In [5]:
results, highest_test_accuracy_index, highest_train_accuracy_index, highest_test_precision_index, highest_test_recall_index= randomGridSearch.fit()

{'activation': 'sigmoid', 'dropout_rate': 0.5, 'filters': 15, 'kernel_size': 3, 'l1_r': 0.1, 'l2_r': 0.001, 'output_layer_activation': 'softmax', 'padding': 'same', 'start_neurons': 128}
softmax
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (64, 3, 15)               270060    
_________________________________________________________________
batch_normalization (BatchNo (64, 3, 15)               60        
_________________________________________________________________
flatten (Flatten)            (64, 45)                  0         
_________________________________________________________________
dense (Dense)                (64, 2)                   92        
Total params: 270,212
Trainable params: 270,182
Non-trainable params: 30
_________________________________________________________________
Starting: 
{'batch_size': 64, 'epochs': 1, 'learning

In [6]:
results_df = randomGridSearch.results_df.copy()

In [7]:
results_df

,batch_size,epochs,learning_rate,optimizer,activation,dropout_rate,filters,kernel_size,l1_r,l2_r,...,padding,start_neurons,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall
0,64,1,0.001,rmsprop,sigmoid,0.5,15,3,0.1,0.001,...,same,128,40.506882,0.539522,0.539522,0.539522,40.731785,0.458333,0.458333,0.458333
1,8,1,1e-05,sgd,softmax,0.2,21,7,0.2,0.1,...,same,512,1008.483215,0.541058,0.541058,0.541058,1008.476318,0.557292,0.557292,0.557292


In [8]:
"""metrics_train = {"train_loss" : 0, "train_accuracy" : 1, "train_precision" : 2, "train_recall" : 4}
metrics_val = {"val_loss" : 0, "val_accuracy" : 1, "val_precision" : 2, "val_recall" : 4}
finished_train = False
# Get last row in results
temp_df = results_df.tail(1).copy()
for idx, column in enumerate(temp_df.columns):
    if idx >= 13:
        temp_df[column] = temp_df[column].astype('float')
# Get list of columns containing nan values
#unfinished_columns = results_df.columns[results_df.isnull().any()].tolist()
unfinished_columns = results_df.columns[temp_df.isnull().any()].tolist()
print(unfinished_columns)
for idx, column in enumerate(unfinished_columns):
    if not finished_train:
        # Change value at column using the metrics
        results_df.iloc[-1, results_df.columns.get_loc(column)] = metrics_train[column]
        if idx == len(metrics_train)-1:
            finished_train = True

    else:
        # Change value at column using the metrics
        results_df.iloc[-1, results_df.columns.get_loc(column)] = metrics_val[column]
"""

'metrics_train = {"train_loss" : 0, "train_accuracy" : 1, "train_precision" : 2, "train_recall" : 4}\nmetrics_val = {"val_loss" : 0, "val_accuracy" : 1, "val_precision" : 2, "val_recall" : 4}\nfinished_train = False\n# Get last row in results\ntemp_df = results_df.tail(1).copy()\nfor idx, column in enumerate(temp_df.columns):\n    if idx >= 13:\n        temp_df[column] = temp_df[column].astype(\'float\')\n# Get list of columns containing nan values\n#unfinished_columns = results_df.columns[results_df.isnull().any()].tolist()\nunfinished_columns = results_df.columns[temp_df.isnull().any()].tolist()\nprint(unfinished_columns)\nfor idx, column in enumerate(unfinished_columns):\n    if not finished_train:\n        # Change value at column using the metrics\n        results_df.iloc[-1, results_df.columns.get_loc(column)] = metrics_train[column]\n        if idx == len(metrics_train)-1:\n            finished_train = True\n\n    else:\n        # Change value at column using the metrics\n      

In [9]:
results_df

,batch_size,epochs,learning_rate,optimizer,activation,dropout_rate,filters,kernel_size,l1_r,l2_r,...,padding,start_neurons,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall
0,64,1,0.001,rmsprop,sigmoid,0.5,15,3,0.1,0.001,...,same,128,40.506882,0.539522,0.539522,0.539522,40.731785,0.458333,0.458333,0.458333
1,8,1,1e-05,sgd,softmax,0.2,21,7,0.2,0.1,...,same,512,1008.483215,0.541058,0.541058,0.541058,1008.476318,0.557292,0.557292,0.557292


In [10]:
ex = {"A" : 1,
      "B" : 2}
print(ex["A"])

keys = list(hyper_grid.keys())
keys2 = list(model_grid.keys())

1


In [11]:
print(keys)
print(keys2)

['batch_size', 'epochs', 'learning_rate', 'optimizer']
['activation', 'dropout_rate', 'filters', 'kernel_size', 'l1_r', 'l2_r', 'output_layer_activation', 'padding', 'start_neurons']


In [12]:
keys[0]

'batch_size'

In [13]:
import numpy as np
import pandas as pd
hyper_keys = list(hyper_grid.keys())
model_keys = list(model_grid.keys())
metrics_train_keys = ["train_loss", "train_accuracy", "train_presicion", "train_recall"]
metrics_val_keys = ["val_loss", "val_accuracy", "val_presicion", "val_recall"]
header = np.concatenate((hyper_keys, model_keys, metrics_train_keys, metrics_val_keys))
info_table = pd.DataFrame(np.array([i for i in range(len(header))]).reshape(1, 21) ,columns = header)

In [14]:
info_table

,batch_size,epochs,learning_rate,optimizer,activation,dropout_rate,filters,kernel_size,l1_r,l2_r,...,padding,start_neurons,train_loss,train_accuracy,train_presicion,train_recall,val_loss,val_accuracy,val_presicion,val_recall
0,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20


In [15]:
testing = pd.DataFrame([[0, np.nan]], columns = ["a", "b"])

In [16]:
to_add = pd.DataFrame(np.array([0 ,1]).reshape(1,2), columns = ["a", "b"])
print(to_add)
testing = testing.append(to_add)
testing

   a  b
0  0  1


,a,b
0,0,NaN
0,0,1.0


In [17]:
nan_columns = testing.columns[testing.isnull().any()].tolist()

In [18]:
temp_df

NameError: name 'temp_df' is not defined

In [ ]:
nan_columns = temp_df.columns[temp_df.isna().any()].tolist()

In [ ]:
nan_cols = [i for i in temp_df.columns if pd.isnull(temp_df[i].values)]

In [ ]:
for column in temp_df.columns:
    print(temp_df[column].values)

In [ ]:
temp_df['val_loss'].dtype